# Storage Access API Demo Analysis

This jupyter notebook can be used to extract the results of the SAA experiment from the database and analyze it.

In [ ]:
# Automatically reload all modules that are changed before executing code
%load_ext autoreload
%autoreload 2

## General Analysis of Tasks

In [ ]:
from database import Task, URL
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML

In [ ]:
# Select data from db
tasks = Task.select().where(Task.job == "storageaccessapi")
urls = URL.select().where(URL.job == "storageaccessapi")

# Transform to dataframes
tasks = pd.DataFrame(list(tasks.dicts()))
urls = pd.DataFrame(list(urls.dicts()))

# Show tasks table
display(tasks)